In [2]:
import requests
import pandas as pd
import geojson
from shapely.geometry import shape
from shapely.geometry import Point
import json as js

from dotenv import load_dotenv
import os

Here we use a .env file to ensure that the API key does not go public  
However, the API is free to use, you just have to apply [here](https://info.nobil.no/api) to get  
the key. If you don't want to apply there is also a client test of the  
api [here](https://www.nobil.no/api/client/search_apiVer3.php)

In [3]:
# Collect the api key
load_dotenv()

API_KEY = os.environ.get('API_KEY')

In [4]:
# Ne stands for the coordinates to the northeast most point where you want to search
# Sw stands for the southwestern most point you want to search.
# Together they form a square where you ask the API to send back data
def get_api_data(ne,sw,API_KEY):
    url = 'https://nobil.no/api/server/search.php?mode=ajax'
    post_data = { 'apikey': f'{API_KEY}', 'apiversion': '3',
    'action': "search",
    'type': 'rectangle',
    'northeast': f'{ne}',
    'southwest': f'{sw}'}
    return requests.post(url, data=post_data)

def json_to_file(file,filename):
    with open(f"{filename}.json", "w") as write_file:
        js.dump(file, write_file)

In [5]:
def get_pol(path):
    # Opening JSON file
    with open(path,'r') as file:
        # returns JSON object as
        # a dictionary
        gj = geojson.load(file)
    features = gj['features'][0]
    geom = features['geometry']
    return shape(geom)

Quickly sketch out a bounding box for sweden in geojson [here](www.geojson.io)  
Then we can take the max coordinates to find the min / max | lat / lon   
Then we get the Southwest / Northeast coords we need for the API query

In [6]:
p = get_pol('map.geojson')

lats,lons = [],[]

for c in p.coords:
    lon = c[0]
    lat = c[1]
    lats.append(lat)
    lons.append(lon)

ne = max(lats),max(lons)
sw = min(lats),min(lons)